In [10]:
import pandas as pd
import numpy as np
from datetime import datetime


In [11]:
# Load clean data
df = pd.read_csv('../data/processed/loan_data_clean.csv')
df['issue_date'] = pd.to_datetime(df['issue_date'])

In [12]:
# Cell 2: CREATE MAIN DASHBOARD DATASET
print("📊 Creating main dashboard dataset...")

# Main dataset with all necessary fields for Power BI
dashboard_main = df[[
    'id', 'address_state', 'emp_length', 'grade', 'sub_grade',
    'home_ownership', 'issue_date', 'loan_status', 'loan_category',
    'purpose', 'term', 'annual_income', 'dti', 'installment', 
    'int_rate', 'loan_amount', 'total_payment',
    'issue_year', 'issue_month', 'issue_month_name',
    'income_bracket', 'dti_category'
]].copy()

# Add calculated fields for Power BI
dashboard_main['recovery_rate'] = (dashboard_main['total_payment'] / dashboard_main['loan_amount']) * 100
dashboard_main['profit_loss'] = dashboard_main['total_payment'] - dashboard_main['loan_amount']
dashboard_main['monthly_income'] = dashboard_main['annual_income'] / 12
dashboard_main['loan_to_income_ratio'] = dashboard_main['loan_amount'] / dashboard_main['annual_income']

print(f"Main dashboard dataset: {len(dashboard_main)} records, {len(dashboard_main.columns)} columns")
dashboard_main.to_csv('../data/exports/dashboard_main.csv', index=False)

📊 Creating main dashboard dataset...
Main dashboard dataset: 35274 records, 26 columns


In [13]:
# Cell 3: CREATE KPI SUMMARY TABLE
print("📈 Creating KPI summary for Power BI cards...")

# Calculate all KPIs in a structured format for Power BI cards
kpi_cards = []

# Primary KPIs
kpi_cards.extend([
    {'KPI_Category': 'Volume', 'KPI_Name': 'Total Loan Applications', 'KPI_Value': len(df), 'KPI_Format': 'Number'},
    {'KPI_Category': 'Amount', 'KPI_Name': 'Total Funded Amount', 'KPI_Value': df['loan_amount'].sum(), 'KPI_Format': 'Currency'},
    {'KPI_Category': 'Amount', 'KPI_Name': 'Total Amount Received', 'KPI_Value': df['total_payment'].sum(), 'KPI_Format': 'Currency'},
    {'KPI_Category': 'Rate', 'KPI_Name': 'Average Interest Rate', 'KPI_Value': df['int_rate'].mean(), 'KPI_Format': 'Percentage'},
    {'KPI_Category': 'Risk', 'KPI_Name': 'Average DTI', 'KPI_Value': df['dti'].mean(), 'KPI_Format': 'Percentage'}
])

# Good vs Bad Loan KPIs
good_loans = df[df['loan_category'] == 'Good Loan']
bad_loans = df[df['loan_category'] == 'Bad Loan']

kpi_cards.extend([
    {'KPI_Category': 'Quality', 'KPI_Name': 'Good Loan Percentage', 'KPI_Value': (len(good_loans)/len(df))*100, 'KPI_Format': 'Percentage'},
    {'KPI_Category': 'Quality', 'KPI_Name': 'Good Loan Applications', 'KPI_Value': len(good_loans), 'KPI_Format': 'Number'},
    {'KPI_Category': 'Quality', 'KPI_Name': 'Good Loan Funded Amount', 'KPI_Value': good_loans['loan_amount'].sum(), 'KPI_Format': 'Currency'},
    {'KPI_Category': 'Quality', 'KPI_Name': 'Bad Loan Percentage', 'KPI_Value': (len(bad_loans)/len(df))*100, 'KPI_Format': 'Percentage'},
    {'KPI_Category': 'Quality', 'KPI_Name': 'Bad Loan Applications', 'KPI_Value': len(bad_loans), 'KPI_Format': 'Number'},
    {'KPI_Category': 'Quality', 'KPI_Name': 'Bad Loan Funded Amount', 'KPI_Value': bad_loans['loan_amount'].sum(), 'KPI_Format': 'Currency'}
])

kpi_summary_df = pd.DataFrame(kpi_cards)
kpi_summary_df.to_csv('../data/exports/kpi_summary_cards.csv', index=False)
print(f"KPI summary created: {len(kpi_summary_df)} KPIs")


📈 Creating KPI summary for Power BI cards...
KPI summary created: 11 KPIs


In [14]:
# Cell 4: CREATE TIME SERIES DATA FOR TRENDS
print("📅 Creating time series data for trend analysis...")

# Monthly aggregation for trend charts
monthly_trends = df.groupby(['issue_year', 'issue_month', 'issue_month_name']).agg({
    'id': 'count',
    'loan_amount': 'sum',
    'total_payment': 'sum',
    'int_rate': 'mean',
    'dti': 'mean'
}).reset_index()

monthly_trends.columns = ['Year', 'Month_Number', 'Month_Name', 'Total_Applications', 
                         'Total_Funded', 'Total_Received', 'Avg_Interest_Rate', 'Avg_DTI']

# Create date field for proper sorting in Power BI
monthly_trends['Date'] = pd.to_datetime(
    monthly_trends.rename(columns={'Year':'year','Month_Number':'month'})[['year','month']].assign(day=1))
monthly_trends = monthly_trends.sort_values('Date')

# Add running totals
monthly_trends['Cumulative_Applications'] = monthly_trends['Total_Applications'].cumsum()
monthly_trends['Cumulative_Funded'] = monthly_trends['Total_Funded'].cumsum()
monthly_trends['Cumulative_Received'] = monthly_trends['Total_Received'].cumsum()

monthly_trends.to_csv('../data/exports/monthly_trends_detailed.csv', index=False)
print(f"Monthly trends data: {len(monthly_trends)} months")


📅 Creating time series data for trend analysis...
Monthly trends data: 12 months


In [15]:
# Cell 5: CREATE DIMENSIONAL TABLES FOR POWER BI
print("🏗️ Creating dimensional tables for Power BI relationships...")

# State dimension table
state_dim = df.groupby('address_state').agg({
    'id': 'count',
    'loan_amount': ['sum', 'mean'],
    'total_payment': ['sum', 'mean'],
    'int_rate': 'mean'
}).round(2)

state_dim.columns = ['Total_Apps', 'Total_Funded', 'Avg_Loan_Amount', 'Total_Received', 'Avg_Received', 'Avg_Interest_Rate']
state_dim.reset_index(inplace=True)
state_dim.to_csv('../data/exports/state_dimension.csv', index=False)

# Grade dimension table  
grade_dim = df.groupby(['grade', 'sub_grade']).agg({
    'id': 'count',
    'loan_amount': ['sum', 'mean'],
    'int_rate': 'mean',
    'dti': 'mean'
}).round(2)

grade_dim.columns = ['Total_Apps', 'Total_Funded', 'Avg_Loan_Amount', 'Avg_Interest_Rate', 'Avg_DTI']
grade_dim.reset_index(inplace=True)
grade_dim.to_csv('../data/exports/grade_dimension.csv', index=False)

# Employment dimension table
emp_dim = df.groupby('emp_length').agg({
    'id': 'count',
    'loan_amount': ['sum', 'mean'],
    'annual_income': 'mean',
    'dti': 'mean',
    'int_rate': 'mean'
}).round(2)

emp_dim.columns = ['Total_Apps', 'Total_Funded', 'Avg_Loan_Amount', 'Avg_Annual_Income', 'Avg_DTI', 'Avg_Interest_Rate']
emp_dim.reset_index(inplace=True)
emp_dim.to_csv('../data/exports/employment_dimension.csv', index=False)

print("✅ Dimensional tables created")


🏗️ Creating dimensional tables for Power BI relationships...
✅ Dimensional tables created


In [16]:
# Cell 6: CREATE POWER BI RELATIONSHIP SCHEMA
print("🔗 Creating Power BI relationship schema documentation...")

relationships_info = {
    'Table_Name': [
        'dashboard_main',
        'kpi_summary_cards', 
        'monthly_trends_detailed',
        'state_dimension',
        'grade_dimension', 
        'employment_dimension'
    ],
    'Primary_Key': [
        'id',
        'KPI_Name',
        'Date',
        'address_state',
        'grade, sub_grade',
        'emp_length'
    ],
    'Purpose': [
        'Main fact table with all loan records',
        'KPI values for dashboard cards',
        'Time series data for trend analysis',
        'State-level aggregated metrics',
        'Grade/Sub-grade aggregated metrics',
        'Employment length aggregated metrics'
    ],
    'Relationships': [
        'Central fact table',
        'Standalone for KPI cards',
        'Filter by date range',
        'Join on address_state',
        'Join on grade/sub_grade',
        'Join on emp_length'
    ]
}

relationship_schema = pd.DataFrame(relationships_info)
relationship_schema.to_csv('../data/exports/powerbi_schema.csv', index=False)
print(relationship_schema.to_string(index=False))


🔗 Creating Power BI relationship schema documentation...
             Table_Name      Primary_Key                               Purpose            Relationships
         dashboard_main               id Main fact table with all loan records       Central fact table
      kpi_summary_cards         KPI_Name        KPI values for dashboard cards Standalone for KPI cards
monthly_trends_detailed             Date   Time series data for trend analysis     Filter by date range
        state_dimension    address_state        State-level aggregated metrics    Join on address_state
        grade_dimension grade, sub_grade    Grade/Sub-grade aggregated metrics  Join on grade/sub_grade
   employment_dimension       emp_length  Employment length aggregated metrics       Join on emp_length


In [17]:
# Cell 7: FINAL DATA VALIDATION AND EXPORT SUMMARY
print("\n🔍 Final validation of Power BI exports...")

export_files = [
    'dashboard_main.csv',
    'kpi_summary_cards.csv', 
    'monthly_trends_detailed.csv',
    'state_dimension.csv',
    'grade_dimension.csv',
    'employment_dimension.csv',
    'powerbi_schema.csv'
]

print("\n📂 POWER BI EXPORT SUMMARY:")
print("="*50)

for file in export_files:
    file_path = f'../data/exports/{file}'
    try:
        df_temp = pd.read_csv(file_path)
        print(f"✅ {file}: {len(df_temp)} records, {len(df_temp.columns)} columns")
    except FileNotFoundError:
        print(f"❌ {file}: File not found")

print("\n🎯 POWER BI DASHBOARD STRUCTURE READY:")
print("1. Summary Dashboard - Use kpi_summary_cards.csv")
print("2. Trends Dashboard - Use monthly_trends_detailed.csv") 
print("3. Regional Analysis - Use state_dimension.csv")
print("4. Loan Quality Analysis - Use dashboard_main.csv")
print("5. Detailed Records - Use dashboard_main.csv")

print("\n✅ All data prepared for Power BI import!")
print("Next: Import these CSV files into Power BI and create relationships as per powerbi_schema.csv")



🔍 Final validation of Power BI exports...

📂 POWER BI EXPORT SUMMARY:
✅ dashboard_main.csv: 35274 records, 26 columns
✅ kpi_summary_cards.csv: 11 records, 4 columns
✅ monthly_trends_detailed.csv: 12 records, 12 columns
✅ state_dimension.csv: 50 records, 7 columns
✅ grade_dimension.csv: 35 records, 7 columns
✅ employment_dimension.csv: 11 records, 7 columns
✅ powerbi_schema.csv: 6 records, 4 columns

🎯 POWER BI DASHBOARD STRUCTURE READY:
1. Summary Dashboard - Use kpi_summary_cards.csv
2. Trends Dashboard - Use monthly_trends_detailed.csv
3. Regional Analysis - Use state_dimension.csv
4. Loan Quality Analysis - Use dashboard_main.csv
5. Detailed Records - Use dashboard_main.csv

✅ All data prepared for Power BI import!
Next: Import these CSV files into Power BI and create relationships as per powerbi_schema.csv
